In [88]:
# execute this cell before you start

import tensorflow as tf
from tensorflow.keras import layers

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

print(tf.VERSION)
print(tf.keras.__version__)


1.12.0
2.1.6-tf


#  CA2
## due on 8/03/2019

to submit the assignment, please do the following:

- do `Cell -> All output -> Clear` to clear all your output
- save the notebook (CA3.ipynb)

# The Reuters newswire data

Consider the data in  `keras.datasets.reuters` and train a network which reliably categorizes the newswires. 

In [89]:
reuters = keras.datasets.reuters
num_words=1000
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=num_words)

num_classes = len(set(train_labels))

In [90]:
len(test_data), len(test_labels)

(2246, 2246)

In [92]:
#len(set(train_labels))

In [93]:
word_index = reuters.get_word_index()
word_index["<PAD>"] = 0


reverse_word_index = {}
for key, value in word_index.items() :
    reverse_word_index[value] = key

#reverse_word_index

def decode_article(article):
    decodedArticle = ""
    for word_code in article:
        decodedArticle += " "
        decodedArticle += reverse_word_index.get(word_code, "?")
    return decodedArticle












In [94]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=word_index["<PAD>"], maxlen=256, padding="post")
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=word_index["<PAD>"], maxlen=256, padding="post")

decode_article(train_data[0])

' the of of mln loss for plc said at only ended said of could 1 traders now april 0 a after said from 1985 and from foreign 000 april 0 prices its account year a but in this mln home an states earlier and rise and revs vs 000 its 16 vs 000 a but 3 of of several and shareholders and dividend vs 000 its all 4 vs 000 1 mln agreed of april 0 are 2 states will billion total and against 000 pct dlrs <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PA

In [95]:
'''tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000)
train_data = tokenizer.sequences_to_matrix(train_data, mode='binary')
test_data = tokenizer.sequences_to_matrix(test_data, mode='binary')

train_labels = keras.utils.to_categorical(train_labels, num_classes)
test_labels = keras.utils.to_categorical(test_labels, num_classes)'''

"tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000)\ntrain_data = tokenizer.sequences_to_matrix(train_data, mode='binary')\ntest_data = tokenizer.sequences_to_matrix(test_data, mode='binary')\n\ntrain_labels = keras.utils.to_categorical(train_labels, num_classes)\ntest_labels = keras.utils.to_categorical(test_labels, num_classes)"

In [101]:

vocab_size = num_words
#vocab_size = 10000
#output_layer_size = len(set(train_labels))
model = keras.models.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(512, activation=tf.nn.relu))
model.add(keras.layers.Dense(num_classes, activation=tf.nn.softmax))

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x=train_data, y=train_labels, epochs=40)
test_loss, test_acc = model.evaluate(x=test_data, y=test_labels)
test_loss, test_acc

# the final layer needs to be the size of train_labels ???


/home/neeraj/tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/40
8982/8982 [==============================] - 1s 164us/step - loss: 2.4592 - acc: 0.3560
Epoch 2/40
8982/8982 [==============================] - 1s 74us/step - loss: 1.9235 - acc: 0.4734
Epoch 3/40
8982/8982 [==============================] - 1s 64us/step - loss: 1.6684 - acc: 0.5835
Epoch 4/40
8982/8982 [==============================] - 1s 63us/step - loss: 1.5481 - acc: 0.6123
Epoch 5/40
8982/8982 [==============================] - 1s 63us/step - loss: 1.4592 - acc: 0.6350
Epoch 6/40
8982/8982 [==============================] - 1s 65us/step - loss: 1.3564 - acc: 0.6668
Epoch 7/40
8982/8982 [==============================] - 1s 64us/step - loss: 1.2517 - acc: 0.6887
Epoch 8/40
8982/8982 [==============================] - 1s 65us/step - loss: 1.1758 - acc: 0.7083
Epoch 9/40
8982/8982 [==============================] - 1s 64us/step - loss: 1.1185 - acc: 0.7184
Epoch 10/40
8982/8982 [==============================] - 1s 66us/step - loss: 1.0737 - acc: 0.7305
Epoch 11/40
8982/8

(1.1416884018498028, 0.75289403383793407)

In [8]:
reverse_word_index[min(list(reverse_word_index.keys()))]

'<PAD>'

In [158]:
#reverse_word_index[100]
#train_data[1]
#decode_article(train_data[1])
max(reverse_word_index, key=lambda k:reverse_word_index[k])

23568

In [177]:
reverse_word_index[23568]

'zwermann'